## Deliverable 2. Create a Customer Travel Destinations Map.

In [27]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/zobai/Class/world_weather_analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,East London,ZA,-33.02,27.91,63.97,71,5,15.93,clear sky
1,1,Aginskoye,RU,51.10,114.53,53.49,34,1,4.94,clear sky
2,2,Bayanday,RU,53.07,105.51,49.15,41,3,1.77,clear sky
3,3,Adre,TD,13.47,22.20,83.43,7,88,9.48,overcast clouds
4,4,Khatanga,RU,71.97,102.50,27.82,88,59,5.26,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
# Country column seems to have some null values
preferred_cities_df.count()

City_ID                561
City                   561
Country                557
Lat                    561
Lng                    561
Max Temp               561
Humidity               561
Cloudiness             561
Wind Speed             561
Current Description    561
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# referred to the following documentation to drop by specfic columns:
# https://statisticsglobe.com/drop-rows-blank-values-from-pandas-dataframe-python
preferred_cities_df

# New DataFrame without empty rows
clean_df = preferred_cities_df

clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,East London,ZA,-33.02,27.91,63.97,71,5,15.93,clear sky
1,1,Aginskoye,RU,51.10,114.53,53.49,34,1,4.94,clear sky
3,3,Adre,TD,13.47,22.20,83.43,7,88,9.48,overcast clouds
7,7,Xudat,AZ,41.63,48.68,53.96,74,9,6.44,clear sky
8,8,Katsuura,JP,35.13,140.30,66.60,80,23,4.92,few clouds
...,...,...,...,...,...,...,...,...,...,...
692,692,Pahrump,US,36.21,-115.98,97.84,7,1,3.00,clear sky
693,693,Florianopolis,BR,-27.61,-48.50,67.23,77,0,9.22,clear sky
694,694,Neepawa,CA,50.23,-99.47,70.20,45,88,10.47,overcast clouds
695,695,Abu Zabad,SD,12.35,29.25,85.14,11,48,7.63,scattered clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,East London,ZA,63.97,clear sky,-33.02,27.91,
1,Aginskoye,RU,53.49,clear sky,51.10,114.53,
3,Adre,TD,83.43,overcast clouds,13.47,22.20,
7,Xudat,AZ,53.96,clear sky,41.63,48.68,
8,Katsuura,JP,66.60,few clouds,35.13,140.30,
10,Busselton,AU,53.65,clear sky,-33.65,115.33,
11,Fortuna,US,69.93,clear sky,40.60,-124.16,
14,Tessalit,ML,88.34,light rain,20.20,1.01,
16,Ribeira Grande,PT,64.80,broken clouds,38.52,-28.70,
18,Kapaa,US,80.58,clear sky,22.08,-159.32,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
# Using Paris as my location
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Adding lat/long to location key for the params
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...sk

In [35]:
# 7. Drop the rows where there is no Hotel Name.
# Replace empty values with NaN then drop the NaN values
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
hotel_df = hotel_df.dropna(subset=["Hotel Name"])
hotel_df

C:\Users\zobai\anaconda3\envs\PythonData\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
25,Esperance,AU,57.40,overcast clouds,-33.87,121.90,Hospitality Esperance
33,Sao Filipe,CV,71.78,clear sky,14.90,-24.50,Hotel Xaguate
40,Avarua,CK,82.45,clear sky,-21.21,-159.78,Paradise Inn
46,Cuauhtemoc,MX,81.84,clear sky,28.42,-106.87,Motel Tarahumara Inn
54,Jamestown,US,76.01,clear sky,42.10,-79.24,Hampton Inn & Suites Jamestown
...,...,...,...,...,...,...,...
690,Tailai,CN,55.45,heavy intensity rain,46.38,123.40,Pearl Garden Hotel
691,Krabi,TH,85.05,few clouds,8.07,98.91,Tharapark View Hotel
693,Florianopolis,BR,67.23,clear sky,-27.61,-48.50,Hotel Boutique Quinta das Videiras
694,Neepawa,CA,70.20,overcast clouds,50.23,-99.47,"Westway Inn Motel, Neepawa"


In [36]:
# 8a. Create the output File (CSV)
output_data_file = os.path.join("C:/Users/zobai/Class/world_weather_analysis/Vacation_Search/WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [37]:
# 9. Added hotel name, city name, country code, and the weather description with maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [38]:
# 11a. Add a marker layer for each city to the map.
# 30.0 and 31.0 (lat/long) is the geographic center of the Earth
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))